# Blackjack AI - Entrainement sur Google Colab

Ce notebook rassemble les scripts du depot pour generer les donnees synthetiques et entrainer le detecteur YOLOv8 directement dans un runtime Google Colab.

## Prerequis
- Ouvrez ce notebook dans Google Colab et activez un runtime GPU (Execution > Modifier le type de l environnement d execution).
- Facultatif : montez Google Drive pour sauvegarder les runs et checkpoints.
- Facultatif : preparez vos identifiants Kaggle (`KAGGLE_USERNAME`, `KAGGLE_KEY`) si vous souhaitez importer les jeux de donnees externes.

In [ ]:
import os
import sys
import subprocess
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules
WORKSPACE_ROOT = Path("/content") if IN_COLAB else Path.cwd()
if (WORKSPACE_ROOT / ".git").exists():
    PROJECT_ROOT = WORKSPACE_ROOT
else:
    PROJECT_ROOT = WORKSPACE_ROOT / "BLACKJACK-AI"

print(f"Execution dans Colab : {IN_COLAB}")
print(f"Version Python     : {sys.version.split()[0]}")
print(f"Dossier de travail : {PROJECT_ROOT}")

def run(cmd, cwd=None, check=True):
    if cwd is None:
        cwd = PROJECT_ROOT if PROJECT_ROOT.exists() else None
    pretty = " ".join(str(part) for part in cmd)
    print(f"$ {pretty}")
    return subprocess.run(cmd, cwd=cwd, check=check)

In [ ]:
try:
    gpu_probe = run(["nvidia-smi"], check=False)
    if gpu_probe.returncode != 0:
        print("GPU non detecte ou commande nvidia-smi indisponible.")
except FileNotFoundError:
    print("Commande nvidia-smi introuvable.")

## Sauvegarder les sorties (optionnel)
Activez le montage Google Drive si vous souhaitez conserver les runs et checkpoints hors de la session Colab ephemere.

In [ ]:
MOUNT_DRIVE = False  # passez a True pour enregistrer les sorties dans Google Drive
DRIVE_PATH = Path("/content/drive")
OUTPUT_BASE = None

if MOUNT_DRIVE and IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount(str(DRIVE_PATH))
    OUTPUT_BASE = DRIVE_PATH / "MyDrive" / "blackjack_ai"
    OUTPUT_BASE.mkdir(parents=True, exist_ok=True)
    print(f"Les sorties seront sauvegardees dans {OUTPUT_BASE}")
else:
    print("Montage Drive ignore. Les sorties resteront dans le dossier local.")

## Recuperer le depot

In [ ]:
REPO_URL = "https://github.com/EpicSanDev/BLACKJACK-AI.git"

if PROJECT_ROOT.exists() and (PROJECT_ROOT / ".git").exists():
    print("Depot deja present, on continue.")
else:
    PROJECT_ROOT.parent.mkdir(parents=True, exist_ok=True)
    run(["git", "clone", REPO_URL, str(PROJECT_ROOT)], cwd=PROJECT_ROOT.parent)

os.chdir(PROJECT_ROOT)
print(f"Repertoire actif : {Path.cwd()}")

## Installer les dependances

In [ ]:
BASE_PACKAGES = [
    "ultralytics>=8.1.0",
    "opencv-python-headless",
    "numpy",
    "Flask",
    "mss",
    "requests",
    "pygame",
    "pyyaml",
    "kaggle",
    "tqdm",
]

run(["pip", "install", "--upgrade", "pip"])
run(["pip", "install"] + BASE_PACKAGES)

INSTALL_TORCH = False  # passez a True pour forcer une version CUDA specifique
if INSTALL_TORCH:
    TORCH_INDEX_URL = "https://download.pytorch.org/whl/cu121"
    run([
        "pip",
        "install",
        "torch",
        "torchvision",
        "torchaudio",
        "--index-url",
        TORCH_INDEX_URL,
    ])

In [ ]:
import torch

device = "0" if torch.cuda.is_available() else "cpu"
print(f"torch version : {torch.__version__}")
print(f"CUDA disponible : {torch.cuda.is_available()}")
print(f"Device utilise pour l entrainement : {device}")

## Importer les jeux de donnees Kaggle (optionnel)

In [ ]:
USE_KAGGLE = False  # passez a True puis renseignez vos identifiants
if USE_KAGGLE:
    os.environ["KAGGLE_USERNAME"] = "votre_identifiant"
    os.environ["KAGGLE_KEY"] = "votre_cle"
    print("Identifiants Kaggle charges.")
else:
    print("Kaggle desactive. Modifiez USE_KAGGLE si necessaire.")

In [ ]:
RUN_KAGGLE_STEPS = False  # passez a True pour telecharger et convertir les datasets Kaggle

if RUN_KAGGLE_STEPS:
    if not USE_KAGGLE:
        raise RuntimeError("Activez USE_KAGGLE avant de lancer le telechargement.")
    try:
        run(["python", "tools/download_kaggle_datasets.py", "--force"], cwd=PROJECT_ROOT)
        run(["python", "tools/prepare_kaggle_cards.py", "--overwrite"], cwd=PROJECT_ROOT)
        from tools.one_click_pipeline import determine_blackjack_hands_input
        blackjack_input = determine_blackjack_hands_input()
        if blackjack_input is None:
            raise RuntimeError("Aucun fichier blackjack_hands.* detecte. Verifiez le telechargement Kaggle ou fournissez --input.")
        run(
            [
                "python",
                "tools/prepare_blackjack_hands.py",
                "--sample",
                "0.05",
                "--input",
                str(blackjack_input),
            ],
            cwd=PROJECT_ROOT,
        )
    except subprocess.CalledProcessError as exc:
        print(f"Erreur pendant l'etape Kaggle: {exc}")
        raise
else:
    print("Etapes Kaggle ignorees.")


## Configurer et lancer l entrainement

In [ ]:
TRAIN_NUM_IMAGES = 4000
TRAIN_EPOCHS = 150
TRAIN_BATCH = 24
TRAIN_IMGSZ = 960
TRAIN_NAME = "colab_blackjack"
USE_CACHE = False
REUSE_EXISTING = False  # True pour reutiliser dataset/train et dataset/val existants
ENABLE_AUGMENT = True

runs_base = (OUTPUT_BASE / "runs") if OUTPUT_BASE else (PROJECT_ROOT / "runs")
project_path = runs_base / "detector"
project_path.mkdir(parents=True, exist_ok=True)

train_args = {
    "--num-images": str(TRAIN_NUM_IMAGES),
    "--epochs": str(TRAIN_EPOCHS),
    "--batch": str(TRAIN_BATCH),
    "--imgsz": str(TRAIN_IMGSZ),
    "--weights": str(PROJECT_ROOT / "yolov8n.pt"),
    "--project": str(project_path),
    "--name": TRAIN_NAME,
    "--patience": "60",
    "--workers": "2",
    "--device": device,
}

import torch
if train_args["--device"] != "cpu" and not torch.cuda.is_available():
    print("GPU non detecte, basculement automatique sur le mode CPU.")
    train_args["--device"] = "cpu"

cmd = ["python", "model/train_model.py"]
for key, value in train_args.items():
    if value is None:
        continue
    cmd.extend([key, str(value)])

if USE_CACHE:
    cmd.append("--cache-images")
if not ENABLE_AUGMENT:
    cmd.append("--no-augment")
if REUSE_EXISTING:
    cmd.append("--no-regenerate")
if not RUN_KAGGLE_STEPS:
    cmd.append("--no-kaggle-cards")

train_process = run(cmd, cwd=PROJECT_ROOT)
if train_process.returncode != 0:
    raise RuntimeError("L entrainement s est termine avec un code de retour non nul.")

## Inspecter les resultats

In [ ]:
runs_root = Path(train_args["--project"]).resolve()
if not runs_root.exists():
    raise FileNotFoundError(f"Le dossier des runs est introuvable: {runs_root}")

run_dirs = sorted(
    [p for p in runs_root.iterdir() if p.is_dir()],
    key=lambda p: p.stat().st_mtime,
    reverse=True,
)
if not run_dirs:
    raise RuntimeError("Aucun run detecte dans le dossier selectionne.")

latest_run = run_dirs[0]
print(f"Dernier run : {latest_run}")

results_png = latest_run / "results.png"
if results_png.exists():
    from IPython.display import Image, display

    display(Image(filename=str(results_png)))
else:
    print("Pas de results.png a afficher.")

metrics_csv = latest_run / "results.csv"
if metrics_csv.exists():
    with metrics_csv.open("r", encoding="utf-8") as fh:
        rows = fh.readlines()
    print("Dernieres lignes de results.csv :")
    for line in rows[-10:]:
        print(line.strip())
else:
    print("Pas de results.csv a analyser.")

## Apercu d une inference sur un echantillon

In [ ]:
from ultralytics import YOLO
import random
from IPython.display import Image, display

weights_path = latest_run / "weights" / "best.pt"
if not weights_path.exists():
    raise FileNotFoundError(f"best.pt introuvable dans {latest_run / 'weights'}")

model = YOLO(str(weights_path))

val_dir = PROJECT_ROOT / "dataset" / "val"
candidates = sorted(val_dir.glob("*.png"))
if not candidates:
    raise FileNotFoundError("Aucune image dans dataset/val pour la demonstration.")

sample_path = random.choice(candidates)
print(f"Inference sur : {sample_path}")

preview_dir = latest_run / "preview"
preview_dir.mkdir(parents=True, exist_ok=True)

model.predict(
    source=str(sample_path),
    imgsz=TRAIN_IMGSZ,
    conf=0.25,
    device=device,
    save=True,
    project=str(latest_run),
    name="preview",
    exist_ok=True,
)

preview_images = sorted(preview_dir.glob("*.jpg")) + sorted(preview_dir.glob("*.png"))
if preview_images:
    display(Image(filename=str(preview_images[0])))
else:
    print("Aucune image de preview generee.")